In [36]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [37]:
df = pd.read_csv('ner_dataset - Copy.csv', encoding = "ISO-8859-1")
#df = df[:100000]
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [38]:
import pandas as pd
df = pd.read_csv('ner_dataset - Copy.csv', encoding = "ISO-8859-1")
print(df)

          Sentence #           Word  POS Tag
0        Sentence: 1      Thousands  NNS   O
1                NaN             of   IN   O
2                NaN  demonstrators  NNS   O
3                NaN           have  VBP   O
4                NaN        marched  VBN   O
...              ...            ...  ...  ..
1048570          NaN           they  PRP   O
1048571          NaN      responded  VBD   O
1048572          NaN             to   TO   O
1048573          NaN            the   DT   O
1048574          NaN         attack   NN   O

[1048575 rows x 4 columns]


In [39]:
df = pd.read_csv('ner_dataset.csv', encoding = "ISO-8859-1")
df.isnull().sum()

Sentence #    596361
Word               0
POS                0
Tag                0
dtype: int64

In [40]:
sentence = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter it'
print(sentence)

European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter it


In [41]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(28562, 27593, 17)

In [42]:
ab=' Subha ka bhula agar sham ko wapas ghar aa jaye then we must thank GPS technology'
print(ab)


 Subha ka bhula agar sham ko wapas ghar aa jaye then we must thank GPS technology


In [9]:
df.groupby('Tag').size().reset_index(name='counts')


,Tag,counts
0,B-art,239
1,B-eve,211
2,B-geo,22266
3,B-gpe,9500
4,B-nat,116
5,B-org,11855
6,B-per,10145
7,B-tim,12060
8,I-art,187
9,I-eve,171


In [43]:
from sklearn.feature_extraction import DictVectorizer
X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=True)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((418698, 56197), (418698,))

In [44]:
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
per = SGDClassifier()
per.fit(X_train,y_train)

SGDClassifier()

In [45]:
rs=per.predict(X_test)

In [46]:
rs

array(['O', 'O', 'O', ..., 'O', 'O', 'O'], dtype='<U5')

In [47]:
y_test[:20]

array(['O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O',
       'I-org', 'O', 'B-geo', 'O', 'I-org', 'O', 'O', 'O', 'B-gpe'],
      dtype=object)

In [48]:
from sklearn.metrics import classification_report

In [49]:
print(classification_report(y_test, rs ))

              precision    recall  f1-score   support

       B-art       0.53      0.11      0.19        79
       B-eve       0.61      0.23      0.33        75
       B-geo       0.66      0.89      0.76      7251
       B-gpe       0.94      0.73      0.82      3195
       B-nat       0.43      0.27      0.33        37
       B-org       0.65      0.51      0.57      3922
       B-per       0.77      0.58      0.66      3378
       B-tim       0.90      0.66      0.76      3998
       I-art       0.50      0.07      0.12        61
       I-eve       0.75      0.09      0.16        65
       I-geo       0.71      0.56      0.63      1436
       I-gpe       1.00      0.02      0.04        44
       I-nat       0.00      0.00      0.00        10
       I-org       0.74      0.51      0.61      3242
       I-per       0.65      0.72      0.69      3409
       I-tim       0.69      0.01      0.01      1244
           O       0.98      1.00      0.99    174779

    accuracy              

In [50]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [51]:

import pandas as pd
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
df = pd.read_csv('ner_dataset - Copy.csv', encoding = "ISO-8859-1")        
getter = SentenceGetter(df)
sentences = getter.sentences

In [52]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [53]:
from sklearn.model_selection import train_test_split
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)


In [55]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)


AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

In [54]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

NameError: name 'new_classes' is not defined